# Figures for SNIa NIR Space Astro 2020 Paper
Michael Wood-Vasey

Figures to Make:
1. SED with UV, optical, NIR filters at z= 0, 0.2, 0.5, 1.0, 1.5
2. Cosmological constraints from reduced intrinsic scatter.
3. Constraints on dust extinction from UV+opt+NIR data.

In [ ]:
from copy import copy

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import sncosmo
from astropy.cosmology import Planck15
import astropy.units as u
import astropy.constants as const

In [ ]:
model = sncosmo.Model('hsiao')
wavelength = np.arange(model.minwave(), model.maxwave(), 10)  # Angstroms
phases = np.array([-19, -10, 0, 10])

In [ ]:
model.param_names

In [ ]:
model.update

In [ ]:
z_0 = Planck15.H0 * (10 * u.pc) / const.c
z_0 = z_0.decompose()

In [ ]:
mu = Planck15.distmod(z_0)

In [ ]:
mu / u.mag

In [ ]:
def plot_model(untouch_model, ax, phases=[1, 10, 20, 50], redshifts=[0], scaling=None,
              wavelength_plot_buffer=[5000, 3000]):
    model = copy(untouch_model)
    for d in phases:
        z_eps = (Planck15.H0 * (10 * u.pc) / const.c).decompose()  # Redshift of 10 pc
        mu = Planck15.distmod(np.asarray(redshifts)+z_eps)  # Enforce a minimum (z_eps ~= 3e-9) so that the naive z=0 doesn't crash
        if scaling is None:
            scaling = 10**(-0.4*(mu/u.mag)).value

        for z, s in zip(redshifts, scaling):
            model.set(z=z)
            this_wavelength = np.arange(model.minwave(), model.maxwave(), 1)
            flux = model.flux(time=d, wave=this_wavelength)
            # Rescale by mu
            flux *= s
            ax.plot(this_wavelength, flux, label='z={}, {} days'.format(z, d))
    
    ax.set_xlim(wavelength[0] - wavelength_plot_buffer[0],
                wavelength[-1] + wavelength_plot_buffer[1])
    ax.set_xlabel(r'Wavelength [$\AA$]')
    ax.set_ylabel(r'flux density [d flux/d$\lambda$]') 
    ax.legend(loc='upper left')

In [ ]:
def overlay_bandpasses(ax,
                       bands=('cspu', 'cspb', 'cspg', 'cspr', 'cspi', 'cspys', 'cspjs', 'csphs', 'cspk'),
                       wavelength=wavelength):
    ax2 = ax.twinx()
    for b in bands:
        band = sncosmo.get_bandpass(b)
        ax2.plot(wavelength, band(wavelength), label=b, linestyle=':')
        
    ax2.legend(loc='upper right')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_model(model, axes[0])

plot_model(model, axes[1])
axes[1].set_yscale('log')
#axes[1].set_ylim(1e-12, 1.2e-8)

overlay_bandpasses(axes[0])
overlay_bandpasses(axes[1])

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.gca()
kwargs = {'phases': [-10], 'redshifts': [0, 0.2, 0.5, 1.0, 1.5], 'scaling': [1e0, 1e-1, 1e-2, 1e-3, 1e-4]}

plot_model(model, ax, **kwargs)

ax.set_yscale('log')
ax.set_ylim(1e-14, 1.2e-8)

overlay_bandpasses(ax)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.gca()
kwargs = {'phases': [0], 'redshifts': [0, 0.2, 0.5, 1.0, 1.5], 'scaling': [1e0, 1e-1, 1e-2, 1e-3, 1e-4]}

plot_model(model, ax, **kwargs)

ax.set_yscale('log')
ax.set_ylim(1e-14, 1.2e-8)

overlay_bandpasses(ax)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.gca()
kwargs = {'phases': [20], 'redshifts': [0, 0.2, 0.5, 1.0, 1.5], 'scaling': [1e0, 1e-1, 1e-2, 1e-3, 1e-4]}

plot_model(model, ax, **kwargs)

ax.set_yscale('log')
ax.set_ylim(1e-14, 1.2e-8)

overlay_bandpasses(ax)